In [12]:
from moviepy.editor import VideoFileClip
from pathlib import Path
import os
import numpy as np
import pandas as pd
import random

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import keras
import keras_cv

from sklearn.preprocessing import LabelEncoder
from sklearn import ensemble
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import matplotlib.pyplot as plt
import seaborn as sns

import shutil
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore", category=UserWarning) #used to supress the tf version warning. 

mids_dir = Path("D:\\MIDS-W207")
data = mids_dir/"MIDS-W207-Spring24-Soccer-Detection-Data"
project = mids_dir/"MIDS-W207-Spring24-Soccer-Detection"
analysis = project/"analysis"

# Author: Timothy Majidzadeh
# Date Created: March 4, 2024
# Date Updated: March 5, 2024
# Description: Work to match the labels data to the images.
# Notes: [v1] Created program.
# Inputs: Frame-by-frame image data & labels.
# Outputs: Object detection model?

In [2]:
labels = pd.read_pickle(data/"base/top_view_labels/top_view_labels.pkl")

In [3]:
labels.head()

vidname frame                 frame_imgname  \
                                                                
                                                                
0  D_20220220_1_0000_0030     1  D_20220220_1_0000_0030_1.png   
1  D_20220220_1_0000_0030     2  D_20220220_1_0000_0030_2.png   
2  D_20220220_1_0000_0030     3  D_20220220_1_0000_0030_3.png   
3  D_20220220_1_0000_0030     4  D_20220220_1_0000_0030_4.png   
4  D_20220220_1_0000_0030     5  D_20220220_1_0000_0030_5.png   

                                       frame_imgpath    team_0          \
                                                     player_00           
                                                     bb_height bb_left   
0  D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      39.0  801.00   
1  D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      39.0  801.75   
2  D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      39.0  802.50   
3  D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      39.0  803.25   
4  D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      39.0  804.00   

                                       ...    team_1                         \
                    player_01          ... player_08 player_09                
    bb_top bb_width bb_height bb_left  ...  bb_width bb_height      bb_left   
0  1051.00     39.0      39.0  1464.0  ...      39.0      39.0  2537.000000   
1  1052.25     39.0      39.0  1464.0  ...      39.0      39.0  2536.857143   
2  1053.50     39.0      39.0  1464.0  ...      39.0      39.0  2536.714286   
3  1054.75     39.0      39.0  1464.0  ...      39.0      39.0  2536.571429   
4  1056.00     39.0      39.0  1464.0  ...      39.0      39.0  2536.428571   

                             ball                            frame_saved  
                             ball                                         
        bb_top bb_width bb_height  bb_left   bb_top bb_width              
0  1146.000000     39.0     10.00  1916.00  1073.00    10.00        True  
1  1145.714286     39.0      9.75  1910.75  1073.75     9.75        True  
2  1145.428571     39.0      9.50  1905.50  1074.50     9.50        True  
3  1145.142857     39.0      9.25  1900.25  1075.25     9.25        True  
4  1144.857143     39.0      9.00  1895.00  1076.00     9.00        True  

[5 rows x 97 columns]

In [4]:
for team in ["team_0", "team_1"]:
    for player in ["player_00", "player_01","player_02","player_03","player_04",
                   "player_05","player_06","player_07","player_08","player_09","player_10"]:
        for bb_dat in ['bb_left', 'bb_top']:
            print("{}, {}, {}:".format(team, player, bb_dat), np.max(labels[team][player][bb_dat]))

team_0, player_00, bb_left: 2610.0
team_0, player_00, bb_top: 1649.0
team_0, player_01, bb_left: 2182.0
team_0, player_01, bb_top: 2046.0
team_0, player_02, bb_left: 3156.0
team_0, player_02, bb_top: 2030.0
team_0, player_03, bb_left: 2756.0
team_0, player_03, bb_top: 2048.0
team_0, player_04, bb_left: 3320.0
team_0, player_04, bb_top: 2018.0
team_0, player_05, bb_left: 3194.0
team_0, player_05, bb_top: 1926.0
team_0, player_06, bb_left: 3478.0
team_0, player_06, bb_top: 1248.0
team_0, player_07, bb_left: 2786.0
team_0, player_07, bb_top: 1844.0
team_0, player_08, bb_left: 2244.0
team_0, player_08, bb_top: 1520.0
team_0, player_09, bb_left: 3066.0
team_0, player_09, bb_top: 1518.0
team_0, player_10, bb_left: 3222.0
team_0, player_10, bb_top: 2039.0
team_1, player_00, bb_left: 3148.0
team_1, player_00, bb_top: 2014.0
team_1, player_01, bb_left: 3254.0
team_1, player_01, bb_top: 2155.0
team_1, player_02, bb_left: 3252.0
team_1, player_02, bb_top: 2050.0
team_1, player_03, bb_left: 3374.0

In [5]:
resized_labels = labels.copy()
resized_labels.loc[
: , (slice(None), slice(None), ['bb_width', 'bb_left']
    )] = round(resized_labels.loc[: , (slice(None), slice(None), ['bb_width', 'bb_left'])] * 1280 / 3840)
resized_labels.loc[
: , (slice(None), slice(None), ['bb_height', 'bb_top'])
] = round(resized_labels.loc[: , (slice(None), slice(None), ['bb_height', 'bb_top'])] * 720 / 2160)

In [6]:
resized_labels

vidname frame                   frame_imgname  \
                                                                    
                                                                    
0    D_20220220_1_0000_0030     1    D_20220220_1_0000_0030_1.png   
1    D_20220220_1_0000_0030     2    D_20220220_1_0000_0030_2.png   
2    D_20220220_1_0000_0030     3    D_20220220_1_0000_0030_3.png   
3    D_20220220_1_0000_0030     4    D_20220220_1_0000_0030_4.png   
4    D_20220220_1_0000_0030     5    D_20220220_1_0000_0030_5.png   
..                      ...   ...                             ...   
895  D_20220220_1_1770_1800   896  D_20220220_1_1770_1800_896.png   
896  D_20220220_1_1770_1800   897  D_20220220_1_1770_1800_897.png   
897  D_20220220_1_1770_1800   898  D_20220220_1_1770_1800_898.png   
898  D_20220220_1_1770_1800   899  D_20220220_1_1770_1800_899.png   
899  D_20220220_1_1770_1800   900  D_20220220_1_1770_1800_900.png   

                                         frame_imgpath    team_0          \
                                                       player_00           
                                                       bb_height bb_left   
0    D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   267.0   
1    D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   267.0   
2    D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   268.0   
3    D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   268.0   
4    D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   268.0   
..                                                 ...       ...     ...   
895  D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   330.0   
896  D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   330.0   
897  D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   330.0   
898  D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   331.0   
899  D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   331.0   

                                       ...    team_1                           \
                    player_01          ... player_08 player_09                  
    bb_top bb_width bb_height bb_left  ...  bb_width bb_height bb_left bb_top   
0    350.0     13.0      13.0   488.0  ...      13.0      13.0   846.0  382.0   
1    351.0     13.0      13.0   488.0  ...      13.0      13.0   846.0  382.0   
2    351.0     13.0      13.0   488.0  ...      13.0      13.0   846.0  382.0   
3    352.0     13.0      13.0   488.0  ...      13.0      13.0   846.0  382.0   
4    352.0     13.0      13.0   488.0  ...      13.0      13.0   845.0  382.0   
..     ...      ...       ...     ...  ...       ...       ...     ...    ...   
895  355.0     16.0      13.0   581.0  ...      13.0      13.0   817.0  455.0   
896  355.0     16.0      13.0   581.0  ...      13.0      13.0   818.0  456.0   
897  355.0     16.0      13.0   581.0  ...      13.0      13.0   819.0  456.0   
898  355.0     16.0      13.0   581.0  ...      13.0      13.0   820.0  457.0   
899  355.0     16.0      13.0   581.0  ...      13.0      13.0   821.0  457.0   

                  ball                         frame_saved  
                  ball                                      
    bb_width bb_height bb_left bb_top bb_width              
0       13.0       3.0   639.0  358.0      3.0        True  
1       13.0       3.0   637.0  358.0      3.0        True  
2       13.0       3.0   635.0  358.0      3.0        True  
3       13.0       3.0   633.0  358.0      3.0        True  
4       13.0       3.0   632.0  359.0      3.0        True  
..       ...       ...     ...    ...      ...         ...  
895     17.0       4.0   913.0  294.0      5.0        True  
896     17.0       4.0   912.0  294.0      5.0        True  
897     17.0       4.0   911.0  294.0      5.0        True  
898     17.0       4.0   910.0  294.0      5.0        True  
899     17.0       4.0   910.0  293.0      5.0        True  

In [7]:
resized_filtered_labels = resized_labels[(resized_labels['frame_saved'] == True)]

In [8]:
resized_filtered_labels

vidname frame                   frame_imgname  \
                                                                    
                                                                    
0    D_20220220_1_0000_0030     1    D_20220220_1_0000_0030_1.png   
1    D_20220220_1_0000_0030     2    D_20220220_1_0000_0030_2.png   
2    D_20220220_1_0000_0030     3    D_20220220_1_0000_0030_3.png   
3    D_20220220_1_0000_0030     4    D_20220220_1_0000_0030_4.png   
4    D_20220220_1_0000_0030     5    D_20220220_1_0000_0030_5.png   
..                      ...   ...                             ...   
895  D_20220220_1_1770_1800   896  D_20220220_1_1770_1800_896.png   
896  D_20220220_1_1770_1800   897  D_20220220_1_1770_1800_897.png   
897  D_20220220_1_1770_1800   898  D_20220220_1_1770_1800_898.png   
898  D_20220220_1_1770_1800   899  D_20220220_1_1770_1800_899.png   
899  D_20220220_1_1770_1800   900  D_20220220_1_1770_1800_900.png   

                                         frame_imgpath    team_0          \
                                                       player_00           
                                                       bb_height bb_left   
0    D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   267.0   
1    D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   267.0   
2    D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   268.0   
3    D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   268.0   
4    D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   268.0   
..                                                 ...       ...     ...   
895  D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   330.0   
896  D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   330.0   
897  D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   330.0   
898  D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   331.0   
899  D:\MIDS-W207\MIDS-W207-Spring24-Soccer-Detecti...      13.0   331.0   

                                       ...    team_1                           \
                    player_01          ... player_08 player_09                  
    bb_top bb_width bb_height bb_left  ...  bb_width bb_height bb_left bb_top   
0    350.0     13.0      13.0   488.0  ...      13.0      13.0   846.0  382.0   
1    351.0     13.0      13.0   488.0  ...      13.0      13.0   846.0  382.0   
2    351.0     13.0      13.0   488.0  ...      13.0      13.0   846.0  382.0   
3    352.0     13.0      13.0   488.0  ...      13.0      13.0   846.0  382.0   
4    352.0     13.0      13.0   488.0  ...      13.0      13.0   845.0  382.0   
..     ...      ...       ...     ...  ...       ...       ...     ...    ...   
895  355.0     16.0      13.0   581.0  ...      13.0      13.0   817.0  455.0   
896  355.0     16.0      13.0   581.0  ...      13.0      13.0   818.0  456.0   
897  355.0     16.0      13.0   581.0  ...      13.0      13.0   819.0  456.0   
898  355.0     16.0      13.0   581.0  ...      13.0      13.0   820.0  457.0   
899  355.0     16.0      13.0   581.0  ...      13.0      13.0   821.0  457.0   

                  ball                         frame_saved  
                  ball                                      
    bb_width bb_height bb_left bb_top bb_width              
0       13.0       3.0   639.0  358.0      3.0        True  
1       13.0       3.0   637.0  358.0      3.0        True  
2       13.0       3.0   635.0  358.0      3.0        True  
3       13.0       3.0   633.0  358.0      3.0        True  
4       13.0       3.0   632.0  359.0      3.0        True  
..       ...       ...     ...    ...      ...         ...  
895     17.0       4.0   913.0  294.0      5.0        True  
896     17.0       4.0   912.0  294.0      5.0        True  
897     17.0       4.0   911.0  294.0      5.0        True  
898     17.0       4.0   910.0  294.0      5.0        True  
899     17.0       4.0   910.0  293.0      5.0        True  

In [45]:
sampled_labels = resized_filtered_labels.sample(frac = 0.15, random_state=507401).reset_index()
bounding_boxes = [
    {
        "classes": [0],
        "boxes": [[
            sampled_labels['ball']['ball']['bb_left'][i],
            sampled_labels['ball']['ball']['bb_top'][i],
            sampled_labels['ball']['ball']['bb_width'][i],
            sampled_labels['ball']['ball']['bb_height'][i]
            ]
        ]
    }
for i in sampled_labels.index]
class_ids = ['ball']

In [46]:
bounding_boxes[0]["boxes"]

[[794.0, 551.0, 6.0, 5.0]]